In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split
from sklearn.metrics import average_precision_score
import numpy as np

In [2]:
df = pd.read_csv("train.csv", index_col="row_id")

xy = df.iloc[:,:2].values
accuracy = df.iloc[:,2].values
time = df.iloc[:,3].values % 1440 # convert to minutes of the day
place_id = df.iloc[:,4].values

In [3]:
neigh = NearestNeighbors(n_jobs=-1, algorithm='kd_tree', leaf_size=30)
%time neigh.fit(xy, place_id)

CPU times: user 1min 56s, sys: 1.22 s, total: 1min 57s
Wall time: 1min 57s


NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [31]:
limit = 400000

subset_indicies = np.random.permutation(len(xy))[:limit]

xy_subset = xy[subset_indicies]
accuracy_subset = accuracy[subset_indicies, None]
time_subset = time[subset_indicies, None]
place_id_subset = place_id[subset_indicies, None]

print("find nearest neighbors")
%time distances_subset, neighbor_indicies_subset = neigh.kneighbors(xy_subset, n_neighbors=200)

find nearest neighbors
CPU times: user 45 s, sys: 7.23 s, total: 52.2 s
Wall time: 14.2 s


In [32]:
# assume accuracy is meters
accuracy_scale = 0.001

def time_difference(time1, time2, period=1440):
    return period/2-np.absolute(np.absolute(time2 - time1) - period/2)

def prob_overlap_time(time1, time2, period=1440):
    return 1 - 2*time_difference(time1, time2)/period

def uniqify(seq):
    seen = set()
    seen_add = seen.add
    return np.fromiter((x for x in seq if not (x in seen or seen_add(x))), dtype=np.int64)

def prob_overlap_locations(dist, accuracy1, accuracy2):
    inv_sumsq = 1 / (np.square(accuracy1) + np.square(accuracy2))
    # if in the end the final result is a product then the 2 * np.pi constant can be removed
    return np.exp(-0.5 * np.square(dist) * inv_sumsq) * inv_sumsq # / (2 * np.pi)

def sum_by_group(values, groups):
    order = np.argsort(groups)
    groups = groups[order]
    values = values[order]
    values.cumsum(out=values)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values = values[index]
    groups = groups[index]
    values[1:] = values[1:] - values[:-1]
    return values, groups

def predict_xy_accuracy_time(distances, neighbor_indicies):
    neighbor_accuracies = accuracy[neighbor_indicies] * accuracy_scale
    test_accuracy = accuracy_subset * accuracy_scale
    neighbor_place_id = place_id[neighbor_indicies]
    colocation_prob = prob_overlap_locations(distances, test_accuracy, neighbor_accuracies)
    
    neighbor_time = time[neighbor_indicies]
    time_prob = prob_overlap_time(time_subset, neighbor_time)
    
    total_prob = colocation_prob * time_prob
    
    # TODO: remove the following line for real data just in case a duplicate point is tested
    s = slice(1,None) if distances[0][0] == 0 else slice(0,None) # skip the first neighbor which will be itself
    predictions = np.zeros((len(distances),3))
    for i, (prob, places) in enumerate(zip(total_prob[:,s], neighbor_place_id[:,s])):
        # append a few zeros just incase there is only one nearby place
        # we need three for the precision calculation
        prob, places = sum_by_group(np.append(prob, [0,0]), np.append(places, [0,0]))
        prob, places = zip(*sorted(zip(prob, places),reverse=True))
        predictions[i,:] = places[:3]
    return predictions
        
def mean_average_precision3(true, test):
    return np.average(np.sum((true == test) * np.array([1, 1/2, 1/3]), axis=1))

print("predict")
%time predictions = predict_xy_accuracy_time(distances_subset, neighbor_indicies_subset)

print("evaluate")
%time mean_average_precision3(place_id_subset, predictions)

predict
CPU times: user 51.2 s, sys: 17.4 s, total: 1min 8s
Wall time: 1min 15s
evaluate
CPU times: user 20.9 ms, sys: 23.6 ms, total: 44.5 ms
Wall time: 87.4 ms


0.56965958333333333

# parameters

* accuracy multiplier
* accuracy bias
* time window in day for full match
* time window in day for least match
* fraction of least match time to full match
* relative weight of time vs distance metric